# Import Libraries

In [1]:
# import standard libraries
import pandas as pd
import numpy as np

# import plotting libraries
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.io as pio
import matplotlib.pyplot as plt
%matplotlib inline

# modeling libraries
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, TimeDistributed, Reshape
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.utils import plot_model

# Pre-Processing & evaluation Libraries
from sklearn.preprocessing import MinMaxScaler, Normalizer, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from scipy.stats import pearsonr as r


# Run Model & load data Functions

In [10]:
def create_timeblock(X, Y, look_back=1):
    dataX, dataY = [], []
    for i in range(len(X) - look_back):
        a = X[i:(i + look_back), :]
        dataX.append(a)
        dataY.append(Y[i + look_back, :])
    print('# of data: ', len(dataY))
    return np.array(dataX), np.array(dataY)


In [11]:
def run_model(x_train, y_train, x_val, y_val, epochs=500, batch_size=400, method='LSTM'):
    classifier = {
                  'LSTM' : LSTM,
                }


    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))

    X = classifier[method](512, return_sequences=True)(inputs)
    X = classifier[method](1024, return_sequences=True)(X)
    X = classifier[method](512, return_sequences=True)(X)
    #X = classifier[method](256, return_sequences=True)(X)
    X = TimeDistributed(Dense(256, activation='relu'))(X)
    X = Dropout(0.4)(X)
    
    out1 = Dense(1, activation='linear')(Flatten()(X))
    out2 = Dense(1, activation='linear')(Flatten()(X))
    out3 = Dense(1, activation='linear')(Flatten()(X))
    
    model = Model(inputs, [out1, out2, out3])
    opt = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.001)

    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['mae'])
    history = model.fit(x_train, [y_train[:, 0], y_train[:, 1], y_train[:, 2]], 
                        epochs=epochs, batch_size=batch_size,
                        validation_data=(x_val, [y_val[:, 0], y_val[:, 1], y_val[:, 2]]), 
                        verbose=1, shuffle=False)
    return model, history

# Load Data

In [12]:
df = pd.read_csv("data\field_data.csv")

# Pre-Processing

In [13]:
val_fraction = 0.15
test_fraction = 0.15
X = df[['AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE',
        'AVG_DP_TUBING', 'AVG_CHOKE_SIZE_P', 'AVG_WHP_P','DP_CHOKE_SIZE']]
Y = df[['BORE_OIL_VOL', 'BORE_GAS_VOL','BORE_WAT_VOL']]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = test_fraction, shuffle = False)

x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train,
                                                  test_size = val_fraction/(1-test_fraction), shuffle = False)

In [14]:
scaler_x = MinMaxScaler()
x_train_norm = scaler_x.fit_transform(x_train.as_matrix())
x_val_norm = scaler_x.transform(x_val.as_matrix())
x_test_norm = scaler_x.transform(X_test.as_matrix())

scaler_y = MinMaxScaler()
y_train_norm = scaler_y.fit_transform(y_train.as_matrix())
y_val_norm = scaler_y.transform(y_val.as_matrix())
y_test_norm = scaler_y.transform(Y_test.as_matrix())

x_train, y_train = create_timeblock(x_train_norm, y_train_norm, look_back=35)
x_val, y_val = create_timeblock(x_val_norm, y_val_norm, look_back=35)
x_test, y_test = create_timeblock(x_test_norm, y_test_norm, look_back=35)

# of data:  1620
# of data:  320
# of data:  320


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



# Training & Validation of the RNN Model 

In [28]:
model_LSTM, history_LSTM = run_model(x_train, y_train,x_val, y_val,  epochs=300, batch_size=400, method='LSTM')

Epoch 1/300
5/5 [==============================] - 28s 6s/step - loss: 14.2267 - dense_9_loss: 5.7711 - dense_10_loss: 5.5413 - dense_11_loss: 2.9142 - dense_9_mae: 1.3584 - dense_10_mae: 1.4252 - dense_11_mae: 1.1280 - val_loss: 0.0373 - val_dense_9_loss: 0.0126 - val_dense_10_loss: 9.5011e-04 - val_dense_11_loss: 0.0237 - val_dense_9_mae: 0.1115 - val_dense_10_mae: 0.0280 - val_dense_11_mae: 0.1478
Epoch 2/300
5/5 [==============================] - 29s 6s/step - loss: 0.1399 - dense_9_loss: 0.0483 - dense_10_loss: 0.0576 - dense_11_loss: 0.0340 - dense_9_mae: 0.1639 - dense_10_mae: 0.1618 - dense_11_mae: 0.1596 - val_loss: 0.0058 - val_dense_9_loss: 1.5594e-04 - val_dense_10_loss: 1.8321e-04 - val_dense_11_loss: 0.0055 - val_dense_9_mae: 0.0100 - val_dense_10_mae: 0.0115 - val_dense_11_mae: 0.0576
Epoch 3/300
5/5 [==============================] - 28s 6s/step - loss: 0.1212 - dense_9_loss: 0.0412 - dense_10_loss: 0.0348 - dense_11_loss: 0.0452 - dense_9_mae: 0.1457 - dense_10_mae: 0.

In [41]:
# Training
y_pred_train_pre = model_LSTM.predict(x_train)
y_pred_train_scaled = np.concatenate((y_pred_train_pre[0], y_pred_train_pre[1], y_pred_train_pre[2]), axis=1)
y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled)
y_train_act = scaler_y.inverse_transform(y_train)

# Dev
y_pred_dev_pre = model_LSTM.predict(x_val)
y_pred_dev_scaled = np.concatenate((y_pred_dev_pre[0], y_pred_dev_pre[1], y_pred_dev_pre[2]), axis=1)
y_pred_dev = scaler_y.inverse_transform(y_pred_dev_scaled)
y_dev_act = scaler_y.inverse_transform(y_val)

# Test
y_pred_test_pre = model_LSTM.predict(x_test)
y_pred_test_scaled = np.concatenate((y_pred_test_pre[0], y_pred_test_pre[1], y_pred_test_pre[2]), axis=1)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled)
y_test_act = scaler_y.inverse_transform(y_test)

# Train-Dev-Test
y_act = np.concatenate([y_train_act, y_dev_act, y_test_act])
y_pred = np.concatenate([y_pred_train, y_pred_dev, y_pred_test])

# Results & Plotting  

In [42]:

def plot_iterative_flow(t, q_actual, q_pred, val_fraction, test_fraction):
    lowrate_type = {0:"Actual field oil rate",
                    1: "Actual field gas rate",
                    2: "Actual field water rate"}
    trace1 = go.Scatter(x = t,
                        y = q_actual,
                        name = lowrate_type[i])
    train_tail = int((1 -val_fraction - test_fraction) * q_pred.shape[0])
    trace2 = go.Scatter(x = t[0:train_tail],
                        y = q_pred[0:train_tail],
                        name = "Training set")

    val_tail = int((1-test_fraction) * q_pred.shape[0])
    trace3 = go.Scatter(x = t[train_tail:val_tail],
                        y = q_pred[train_tail:val_tail],
                        name = "Val set")
    trace4 = go.Scatter(x = t[val_tail:],
                        y = q_pred[val_tail:],
                        name = "Test set")
    data = [trace1, trace2, trace3, trace4]
    layout = {"title": "Flow rate well"}
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig)
    
t = np.arange(len(df))
for i in range(y_act.shape[1]):
    plot_iterative_flow(t,
                   y_act[:, i], y_pred[:, i], val_fraction=val_fraction, test_fraction=test_fraction)
S = 0
cum = []
for i in y_pred_train[:,0]:
    S = S + i
    cum.append(S)
for i in y_pred_dev[:,0]:
    S+= i
    cum.append(S)
for i in y_pred_test[:,0]:
    S+= i
    cum.append(S)
    
T = 0
re = []
for i in y_train_act[:,0]:
    T += i
    re.append(T)
for i in y_dev_act[:,0]:
    T += i
    re.append(T)
for i in y_test_act[:,0]:
    T += i
    re.append(T)
a = np.asarray(cum)
b = np.asarray(re)

trace1 = go.Scatter(x = np.arange(len(a)),
                    y = a.reshape(-1,),
                   name = "Prediction")
trace2 = go.Scatter(x = np.arange(len(a)),
                    y = b.reshape(-1,),
                   name = "Actual values")
data = [trace1, trace2]
layout = go.Layout(title = "Cummulative field oil production of Long short term memory")
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

#GAS
S2 = 0
cum2 = []
for i in y_pred_train[:,1]:
    S2 = S2 + i
    cum2.append(S2)
for i in y_pred_dev[:,1]:
    S2+= i
    cum2.append(S2)
for i in y_pred_test[:,1]:
    S2+= i
    cum2.append(S2)
    
T2 = 0
re2 = []
for i in y_train_act[:,1]:
    T2 += i
    re2.append(T2)
for i in y_dev_act[:,1]:
    T2 += i
    re2.append(T2)
for i in y_test_act[:,1]:
    T2 += i
    re2.append(T2)
a2 = np.asarray(cum2)
b2 = np.asarray(re2)

trace1 = go.Scatter(x = np.arange(len(a2)),
                    y = a2.reshape(-1,),
                   name = "Prediction")
trace2 = go.Scatter(x = np.arange(len(a2)),
                    y = b2.reshape(-1,),
                   name = "Actual values")
data = [trace1, trace2]
layout = go.Layout(title = "Cummulative field gas production of Long short term memory")
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

#WATER
S3 = 0
cum3 = []
for i in y_pred_train[:,2]:
    S3 = S3 + i
    cum3.append(S3)
for i in y_pred_dev[:,2]:
    S3+= i
    cum3.append(S3)
for i in y_pred_test[:,2]:
    S3+= i
    cum3.append(S3)
    
T3 = 0
re3 = []
for i in y_train_act[:,2]:
    T3 += i
    re3.append(T3)
for i in y_dev_act[:,2]:
    T3 += i
    re3.append(T3)
for i in y_test_act[:,2]:
    T3 += i
    re3.append(T3)
a3 = np.asarray(cum3)
b3 = np.asarray(re3)

trace1 = go.Scatter(x = np.arange(len(a3)),
                    y = a3.reshape(-1,),
                   name = "Prediction")
trace2 = go.Scatter(x = np.arange(len(a3)),
                    y = b3.reshape(-1,),
                   name = "Actual values")
data = [trace1, trace2]
layout = go.Layout(title = "Cummulative field water production of Long short term memory")
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)